<a href="https://colab.research.google.com/github/SongYi-Han/BigDataAnalysis/blob/master/final_project_songyi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1. retrieve predicate(relation) and keywords from user query 

more about ner : https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

In [2]:
#fuzzy matching
question = 'who directed Batman 1998'

'''
Who directed The Bridge on the River Kwai?
Who is the director of Good Will Hunting?
Who is the director of Star Wars: Episode VI - Return of the Jedi?
Who is the screenwriter of The Masked Gang: Cyprus?
What is the MPAA film rating of Weathering with You?
What is the genre of Good Neighbors?
'''

question = "Who is the screenwriter of The Masked Gang: Cyprus?"

In [ ]:
## skip this block 

# condition 1 : who vs when/what => assume keywords is either O or S
s_or_o = 'unknown'

q_tokens = question.split()

if q_tokens[0].lower() == 'who':
  s_or_o = 'o'

In [3]:
# remove stopwords   
q_tokens = question.split()
stop_words = set(stopwords.words('english'))
q_tokens = [t for t in q_tokens if not t.lower() in stop_words]


In [4]:
q_tokens
q = " ".join(q_tokens)

In [5]:
nlp = en_core_web_sm.load()
doc = nlp(q)
print([(X, X.ent_iob_, X.ent_type_) for X in doc])

[(screenwriter, 'O', ''), (Masked, 'B', 'PERSON'), (Gang, 'I', 'PERSON'), (:, 'O', ''), (Cyprus, 'O', ''), (?, 'O', '')]


In [6]:
res = [(x.orth_,x.pos_, x.lemma_) for x in [y for y in nlp(question) if not y.is_stop and y.pos_ != 'PUNCT']]
res                                                                      

[('screenwriter', 'NOUN', 'screenwriter'),
 ('Masked', 'PROPN', 'Masked'),
 ('Gang', 'PROPN', 'Gang'),
 ('Cyprus', 'PROPN', 'Cyprus')]

In [9]:
for t in res:
  if t[1] == 'VERB':
    predicate = t[0]

predicate

NameError: ignored

In [8]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
#condition 2 : capture predicate and keywords

if res[0][1] == 'VERB':
  predicate = res[0][2]
  keywords = []
  for r in res[1:] :
    keywords.append(r[2])
  keywords = " ".join(keywords)
  #return predicate, keywords
else: 
  tokens = question.split("of ")
  predicate = [t for t in tokens[0].split() if not t.lower() in stop_words]
  predicate = " ".join(predicate)
  keywords = " ".join(tokens[1:])


In [18]:
## prepared predicate (rleation of movies)
from fuzzywuzzy import process

predicates = ['genre', 'country of origin', 'publication date', 'cast member', 'director', 'based on', 'director', 'screenwriter', 'producer', 'duration']

fuzzy_matching = process.extractOne(predicate, predicates)

if fuzzy_matching[1] > 60 : 
  predicate = fuzzy_matching[0]
else: 
  predicate = None


In [19]:
print(predicate,keywords)

screenwriter The Masked Gang: Cyprus?


In [ ]:
# logic : find predicate and O or S 


# condition 1) find P and keyword
# if question is who/when/where + be + noun + of + keyword (propn)

# if question is who/when/where + verb + keyword 

# condition 2) clarify O or S
# if who => keyword is O, if not => S

# 2. find answer from KG

In [20]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/!_2022_fall/ATAI/Project/code/data

Mounted at /content/gdrive
/content/gdrive/MyDrive/!_2022_fall/ATAI/Project/code/data


In [21]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 25.4 MB/s 
     |████████████████████████████████| 41 kB 477 kB/s 


### load kg

In [22]:
import rdflib

from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import RDFS, SKOS

In [36]:
wd = Namespace('http://www.wikidata.org/entity/')
wdt = Namespace('http://www.wikidata.org/prop/direct/')
wikibase = Namespace('http://wikiba.se/ontology#')
bd =Namespace('http://www.bigdata.com/rdf#')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#label')

In [24]:
graph = rdflib.Graph()
graph.parse('14_graph.nt', format='turtle')

<Graph identifier=Ncc78090c5cc44dc6a70caeddc1ea4ba3 (<class 'rdflib.graph.Graph'>)>

### find item by label

In [ ]:
# https://stackoverflow.com/questions/38527828/how-to-query-wikidata-items-using-its-labels
qres = graph.query(
'''
SELECT ?item ?itemLabel
WHERE { 
  ?item rdfs:label ?itemLabel. 
  FILTER(CONTAINS(LCASE(?itemLabel), "batman"@en)). 
} limit 10
''')

for row in qres:
  print(row.item, row.itemLabel)
  o = row.item

http://www.wikidata.org/entity/Q16531880 Batman Beyond: The Movie
http://www.wikidata.org/entity/Q22249835 Batman v. Superman XXX: An Axel Braun Parody
http://www.wikidata.org/entity/Q2891534 Batman Forever – Original Music From The Motion Picture
http://www.wikidata.org/entity/Q29011652 original script from Batman
http://www.wikidata.org/entity/Q4869347 Batman: Dead White
http://www.wikidata.org/entity/Q10375036 Superman/Batman: Public Enemies
http://www.wikidata.org/entity/Q2121056 Superman/Batman: Public Enemies
http://www.wikidata.org/entity/Q16200150 Batman: Gotham Knights
http://www.wikidata.org/entity/Q176376 The Batman Superman Movie: World's Finest
http://www.wikidata.org/entity/Q2891537 Batman Forever – Original Motion Picture Score Album


In [ ]:
o

rdflib.term.URIRef('http://www.wikidata.org/entity/Q2891537')

In [ ]:
query_template = "SELECT DISTINCT ?x ?y WHERE {{ ?x <{}> <{}>. ?x rdfs:label ?y. }}".format(p, o)

In [53]:
p = 'http://www.wikidata.org/prop/direct/P1001'

query_template = "SELECT ?label WHERE{{<{}> rdfs:label ?label .FILTER(LANG(?label) = 'en'). }}".format(p)
qres = graph.query(query_template)

for row in qres:
  print(row.label)
  p = row.label

applies to jurisdiction


In [ ]:
predicate = "director"

query_template = f"SELECT ?item ?itemLabel WHERE {{?item rdfs:label ?itemLabel. FILTER(CONTAINS(LCASE(?itemLabel), {predicate}@en)). }} limit 10"

qres = graph.query(query_template)

for row in qres:
  print(row.item, row.itemLabel)
  p = row.item

In [38]:
pres = graph.query(
'''
SELECT ?predicate ?label {
    ?predicate a rdf:Property
    OPTIONAL { ?predicate rdfs:label ?label }
} 
''')

for row in qres:
  print(row.item, row.itemLabel)
  p = row.item


http://www.wikidata.org/entity/Q12138255 Michael H – Profession: Director
http://www.wikidata.org/entity/Q18169996 Director's Cut
http://www.wikidata.org/entity/Q29512115 ‘Captain Marvel’ Finds Directors in Anna Boden, Ryan Fleck (EXCLUSIVE)
http://www.wikidata.org/entity/Q3224528 The Wedding Director
http://www.wikidata.org/entity/Q52028617 Daredevil: The Director's Cut
http://www.wikidata.org/prop/direct/P1037 director / manager
http://www.wikidata.org/prop/direct/P3174 art director
http://www.wikidata.org/entity/Q706364 art director
http://www.wikidata.org/prop/direct/P344 director of photography
http://www.wikidata.org/prop/direct/P57 director


In [25]:
qres = graph.query(
'''
SELECT ?item ?itemLabel
WHERE { 
  ?item rdfs:label ?itemLabel. 
  FILTER(CONTAINS(LCASE(?itemLabel), "director"@en)). 
} limit 10
''')

for row in qres:
  print(row.item, row.itemLabel)
  p = row.item

http://www.wikidata.org/entity/Q12138255 Michael H – Profession: Director
http://www.wikidata.org/entity/Q18169996 Director's Cut
http://www.wikidata.org/entity/Q29512115 ‘Captain Marvel’ Finds Directors in Anna Boden, Ryan Fleck (EXCLUSIVE)
http://www.wikidata.org/entity/Q3224528 The Wedding Director
http://www.wikidata.org/entity/Q52028617 Daredevil: The Director's Cut
http://www.wikidata.org/prop/direct/P1037 director / manager
http://www.wikidata.org/prop/direct/P3174 art director
http://www.wikidata.org/entity/Q706364 art director
http://www.wikidata.org/prop/direct/P344 director of photography
http://www.wikidata.org/prop/direct/P57 director


TEST

# Print result

In [ ]:
#prefix wdt: <http://www.wikidata.org/prop/direct/> 
#prefix wd: <http://www.wikidata.org/entity/> 
# ?item wdt:P131 wd:Q72

In [ ]:
query_template = "SELECT DISTINCT ?x ?y WHERE {{ ?x <{}> <{}>. ?x rdfs:label ?y. }}".format(p, o)

print("--- sparql query: {}".format(query_template))

qres = g.query(query_template)

print("\n--- querying results: ")
for row in qres:
    print(row.x, row.y)
    answer = row.y

--- sparql query: SELECT DISTINCT ?x ?y WHERE { ?x <http://www.wikidata.org/prop/direct/P57> <http://www.wikidata.org/entity/Q2891537>. ?x rdfs:label ?y. }

--- querying results: 


In [ ]:
# ?item wdt:P131 wd:Q72
query_template = "SELECT ?item ?label WHERE { ?item <http://www.wikidata.org/prop/direct/P131> <http://www.wikidata.org/entity/Q72>. ?item rdfs:label ?label. }"

qres = graph.query(query_template)

print("\n--- querying results: ")

for row in qres:
  print(row.item, row.label)




--- querying results: 
http://www.wikidata.org/entity/Q692578 Dolder Grand
http://www.wikidata.org/entity/Q222450 Zurich University of the Arts


In [ ]:
#p: director, o: batman vs superman
query_template = "SELECT ?item ?label WHERE { <http://www.wikidata.org/entity/Q2891537> <http://www.wikidata.org/prop/direct/P175> ?item . ?item rdfs:label ?label. }"

qres = graph.query(query_template)

print("\n--- querying results: ")

for row in qres:
  print(row.item, row.label)




--- querying results: 
http://www.wikidata.org/entity/Q264565 Elliot Goldenthal


In [ ]:
answer_template = "Hi, the {} of {} is {}".format(relation, entity, answer)

print("\n--- generated response: {}".format(answer_template))

In [ ]:
query= "SELECT ?s ?sLabel WHERE { ?s rdfs:label "rating"@en . SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } } " 
qres = g.query(query_template)

qres 

# Embedding

In [ ]:
import csv
import numpy as np
import os
import rdflib
import pandas as pd
from sklearn.metrics import pairwise_distances

In [ ]:
# define some prefixes
WD = rdflib.Namespace('http://www.wikidata.org/entity/')
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
DDIS = rdflib.Namespace('http://ddis.ch/atai/')
RDFS = rdflib.namespace.RDFS
SCHEMA = rdflib.Namespace('http://schema.org/')

In [ ]:
# load the embeddings
entity_emb = np.load('entity_embeds.npy')
relation_emb = np.load('relation_embeds.npy')
entity_file = 'entity_ids.del'
relation_file = 'relation_ids.del'

In [ ]:
# load the dictionaries
with open(entity_file, 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open(relation_file, 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}

In [ ]:
ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

In [ ]:
rel2lbl = {rel: str(lbl) for rel, lbl in graph.predicates(RDFS.label)}
lbl2rel = {lbl: ent for ent, lbl in rel2lbl.items()}

In [50]:
predicates = set(graph.predicates())
predicates_url = [str(i.lstrip("rdflib.term.URIRef(")) for i in predicates]

In [93]:
predicates = [rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1001'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P101'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1027'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1029'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P103'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1037'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1038'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1040'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1050'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1056'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P106'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1066'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P108'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1080'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P112'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1142'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1165'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P119'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1196'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P123'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1269'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P127'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1299'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P131'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1327'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1343'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1344'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1346'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P135'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P136'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1365'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1366'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P137'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1376'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P138'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1382'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1399'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P140'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1411'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1412'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1416'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1429'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1431'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1433'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1434'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P144'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1441'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1445'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1455'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P150'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1532'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1542'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P155'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1552'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P156'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P157'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1576'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1589'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P159'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P161'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1657'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P166'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P17'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P170'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P172'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P175'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P178'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P179'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P18'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P180'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1809'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1830'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1875'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1877'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1889'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P19'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P190'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P195'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1981'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1995'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P20'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P205'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P206'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2061'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2079'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P21'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2142'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P22'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2283'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2341'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2348'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2360'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2363'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2408'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P241'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2416'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2438'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P25'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2515'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2541'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2546'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2554'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2563'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P26'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2629'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P263'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2632'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2637'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2650'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2684'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P27'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P272'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2747'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P275'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2756'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2758'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P276'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P279'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2860'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P287'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P291'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P2936'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P30'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P306'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3092'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3095'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P31'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3156'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3174'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3216'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3275'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3300'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3301'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3306'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3342'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3373'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3402'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3428'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P344'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3448'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P345'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P35'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P36'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P361'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P364'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3650'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P366'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P37'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P371'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3803'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3818'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3834'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3842'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3931'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P40'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P400'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P407'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P425'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P437'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P4437'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P449'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P451'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P452'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P4584'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P460'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P4608'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P461'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P462'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P463'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P466'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P47'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P479'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P4805'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P485'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P4908'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P495'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P4969'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P50'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5021'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5028'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5072'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P509'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5096'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5150'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5152'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5201'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P527'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P530'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5389'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P54'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P551'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P57'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P577'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P58'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5940'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5970'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P607'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6166'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6216'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6275'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P629'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6379'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P641'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P66'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6658'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P669'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P674'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6886'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P69'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6942'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P6962'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P7047'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P706'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P710'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P725'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P7327'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P735'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P737'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P740'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P747'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P749'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P750'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P7573'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P767'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P793'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P7936'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P7937'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P7959'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P800'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P802'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P825'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P828'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P832'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P8345'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P8371'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P840'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P8411'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P8670'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P8874'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P8889'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P9086'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P91'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P915'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P921'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P9215'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P931'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P937'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P941'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P945'),
 rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P97')]

In [ ]:
predicates_url = [str(i.lstrip("rdflib.term.URIRef(")) for i in predicates]
predicates_url

In [71]:
len(predicates_url)

251

In [ ]:
rel2lbl = {rel: str(lbl) for rel, lbl in graph.predicates(RDFS.label)}
lbl2rel = {lbl: ent for ent, lbl in rel2lbl.items()}

In [88]:
p = 'http://www.wikidata.org/prop/direct/P101'
query_template = "SELECT ?label WHERE{{<{}> rdfs:label ?label .FILTER(LANG(?label) = 'en'). }}".format(p)
qres = graph.query(query_template)

for row in qres:
  print(row.label)
  p = row.label

field of work


In [66]:
predicates[1]

rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P101')

In [ ]:
rel2lbl = {}
lbl2rel = {}

for p in predicates_url :
  print(p)
  
  query_template = "SELECT ?label WHERE{{<{}> rdfs:label ?label .FILTER(LANG(?label) = 'en'). }}".format(p)
  qres = graph.query(query_template)
  for row in qres:
    lbl = str(row.label)
    print(lbl)

  rel2lbl[p] = lbl
  lbl2rel[lbl] = p
  

In [ ]:
lbl2rel

In [ ]:
# Find the Wikidata ID for the movie (https://www.wikidata.org/wiki/Q132863 is the ID for "Finding Nemo")
# Choose your own movie
movie = WD['Q132863']

# Find the movie in the graph
movie_id = ent2id[movie]

# we compare the embedding of the query entity to all other entity embeddings
distances = pairwise_distances(entity_emb[movie_id].reshape(1, -1), entity_emb, metric='cosine').flatten()

In [ ]:
# Find the Wikidata ID for the movie (https://www.wikidata.org/wiki/Q132863 is the ID for "Finding Nemo")
# Choose your own movie
movie = WD['Q132863']

# Find the movie in the graph
movie_id = ent2id[movie]

# we compare the embedding of the query entity to all other entity embeddings
distances = pairwise_distances(entity_emb[movie_id].reshape(1, -1), entity_emb, metric='cosine').flatten()

# and sort them by distance
most_likely = distances.argsort()

# we print rank, entity ID, entity label, and distance
for rank, idx in enumerate(most_likely[:20]):
    rank = rank + 1
    ent = 'http://www.wikidata.org/entity/Q132863'
    q_id = ent.split('/')[-1] # to convert 'http://www.wikidata.org/entity/Q132863' to 'Q132863'
    lbl = ent2lbl[id2ent[idx]] # eg: 'Finding Nemo'
    dist = distances[idx] # eg: 0.0

    print(f'{rank:2d}. {dist:.3f} {q_id:10s} {lbl}')

 1. 0.000 Q132863    Finding Nemo
 2. 0.163 Q132863    Finding Dory
 3. 0.187 Q132863    Up
 4. 0.199 Q132863    WALL·E
 5. 0.214 Q132863    Toy Story 2
 6. 0.215 Q132863    The Little Mermaid II: Return to the Sea
 7. 0.216 Q132863    A Bug's Life
 8. 0.217 Q132863    Cars
 9. 0.220 Q132863    Cars 2
10. 0.229 Q132863    Toy Story 3
11. 0.230 Q132863    The Good Dinosaur
12. 0.231 Q132863    Monsters, Inc.
13. 0.231 Q132863    Robots
14. 0.232 Q132863    Mary and Max
15. 0.235 Q132863    Monsters University
16. 0.238 Q132863    Maya the Bee
17. 0.238 Q132863    Brave
18. 0.239 Q132863    Inside Out
19. 0.240 Q132863    Finding Nemo – The Musical
20. 0.240 Q132863    Toy Story
